In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import os
import sys
root_folders = ['..', '../..', './']
root_folders = [os.path.abspath(os.path.join(x)) for x in root_folders]
for path in root_folders:
    if path not in sys.path:
        sys.path.append(path)

%load_ext autoreload
%autoreload 2
from deeplavrov.models.wordlevel import AttentionRNNTranslator
from deeplavrov.vocabulary.vocabulary import Index

C:\Users\PC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [38]:
# клетка для того, чтобы создать файлы в нужном формате
def make_pairs_from_anki(path, num_examples=None):
    with open(path, 'r', encoding='utf-8') as f:
        data = f.readlines()
        if num_examples:
            pairs = [x.split('\t') for x in data[:min(len(data), num_examples)]]
        else:
            pairs = [x.split('\t') for x in data]
      
    return [x[0].strip() for x in pairs], [x[1].strip() for x in pairs]

path = os.path.join('rus-eng', 'rus.txt')
num_examples = None
input_lang, target_lang = make_pairs_from_anki(path, num_examples)

from sklearn.model_selection import train_test_split
input_lang_train, input_lang_val, target_lang_train, target_lang_val = train_test_split(input_lang, target_lang, test_size=0.1, shuffle=True)

with open('rus-eng/eng_anki_train.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(input_lang_train).strip())

with open('rus-eng/eng_anki_val.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(input_lang_val).strip())

with open('rus-eng/ru_anki_train.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(target_lang_train).strip())

with open('rus-eng/ru_anki_val.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(target_lang_val).strip())

len(input_lang), len(target_lang)

(336666, 336666)

In [3]:
translator = AttentionRNNTranslator(batch_size=16, num_epochs=13)

In [4]:
translator.fit_from_file(input_file='rus-eng/eng_anki_train.txt', target_file='rus-eng/ru_anki_train.txt')

Building vocab took 23.153021097183228 second(s)
Building vocab took 23.65468144416809 second(s)
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1 Batch 0 Loss 1.6181
Epoch 1 Batch 100 Loss 0.8936
Epoch 1 Batch 200 Loss 1.0823
Epoch 1 Batch 300 Loss 0.7596
Epoch 1 Batch 400 Loss 0.6659
Epoch 1 Batch 500 Loss 0.7487
Epoch 1 Batch 600 Loss 0.7623
Epoch 1 Batch 700 Loss 0.7619
Epoch 1 Batch 800 Loss 0.6047
Epoch 1 Batch 900 Loss 0.6031
Epoch 1 Batch 1000 Loss 0.6449
Epoch 1 Batch 1100 Loss 0.7080
Epoch 1 Batch 1200 Loss 0.7577
Epoch 

Epoch 2 Batch 3600 Loss 0.2606
Epoch 2 Batch 3700 Loss 0.1620
Epoch 2 Batch 3800 Loss 0.2699
Epoch 2 Batch 3900 Loss 0.3320
Epoch 2 Batch 4000 Loss 0.1488
Epoch 2 Batch 4100 Loss 0.2304
Epoch 2 Batch 4200 Loss 0.2511
Epoch 2 Batch 4300 Loss 0.2565
Epoch 2 Batch 4400 Loss 0.1473
Epoch 2 Batch 4500 Loss 0.1774
Epoch 2 Batch 4600 Loss 0.3787
Epoch 2 Batch 4700 Loss 0.1904
Epoch 2 Batch 4800 Loss 0.2569
Epoch 2 Batch 4900 Loss 0.2456
Epoch 2 Batch 5000 Loss 0.2134
Epoch 2 Batch 5100 Loss 0.3293
Epoch 2 Batch 5200 Loss 0.2635
Epoch 2 Batch 5300 Loss 0.2385
Epoch 2 Batch 5400 Loss 0.2486
Epoch 2 Batch 5500 Loss 0.2127
Epoch 2 Batch 5600 Loss 0.2543
Epoch 2 Batch 5700 Loss 0.2427
Epoch 2 Batch 5800 Loss 0.3561
Epoch 2 Batch 5900 Loss 0.2391
Epoch 2 Batch 6000 Loss 0.2143
Epoch 2 Batch 6100 Loss 0.2420
Epoch 2 Batch 6200 Loss 0.3197
Epoch 2 Batch 6300 Loss 0.3472
Epoch 2 Batch 6400 Loss 0.2320
Epoch 2 Batch 6500 Loss 0.1716
Epoch 2 Batch 6600 Loss 0.2341
Epoch 2 Batch 6700 Loss 0.3643
Epoch 2 

KeyboardInterrupt: 

In [21]:
translator.save('model.pickle')

Saved!


In [27]:
translator.translate('hello.')

['привет', '.']

In [6]:
translator.translate('He has a camera.')

['у', 'него', 'есть', 'фотоаппарат', '.']

In [19]:
translator.translate('I was watching television when the telephone rang.')

['я', 'смотрела', 'телевизор', ',', 'когда', 'зазвонил', 'телефон', '.']

In [34]:
translator.translate("Have you heard this joke before?")

['ты', 'уже', 'слышал', 'эту', 'шутку', '?']

In [22]:
t2 = AttentionRNNTranslator.load('model.pickle')

In [23]:
t2.num_epochs, t2.batch_size

(13, 16)

In [28]:
t2.translate('hello.')

['привет', '.']

In [29]:
t2.translate('He has a camera.')

['у', 'него', 'есть', 'фотоаппарат', '.']

In [30]:
t2.translate('I was watching television when the telephone rang.')

['я', 'смотрела', 'телевизор', ',', 'когда', 'зазвонил', 'телефон', '.']

In [33]:
t2.translate("Have you heard this joke before?")

['ты', 'уже', 'слышал', 'эту', 'шутку', '?']

In [43]:
t2.translate("Tom doesn't want to spend the rest of his life in prison.")

['том',
 'не',
 'хочет',
 'провести',
 'остаток',
 'своей',
 'жизни',
 'в',
 'тюрьме',
 '.']